In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from xgboost import XGBClassifier as XGB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

Duplicate key in file WindowsPath('C:/Users/chikachika/anaconda3/Lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 263 ('font.family:  IPAexGothic')


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [5]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [7]:
#Cabinを分割且つ、欠損値補完
train["deck"] = train["Cabin"].apply(lambda x :x.split("/")[0] if isinstance(x, str) else "unknown")
train["num"] = train["Cabin"].apply(lambda x :x.split("/")[1] if isinstance(x, str) else "unknown")
train["side"] = train["Cabin"].apply(lambda x :x.split("/")[2] if isinstance(x, str) else "unknown")

test["deck"] = test["Cabin"].apply(lambda x :x.split("/")[0] if isinstance(x, str) else "unknown")
test["num"] = test["Cabin"].apply(lambda x :x.split("/")[1] if isinstance(x, str) else "unknown")
test["side"] = test["Cabin"].apply(lambda x :x.split("/")[2] if isinstance(x, str) else "unknown")

In [9]:
#Nameを姓と名に分割且つ、欠損値補完
train["first"] = train["Name"].apply(lambda x :x.split(" ")[0] if isinstance(x, str) else "unknown")
train["last"] = train["Name"].apply(lambda x :x.split(" ")[1] if isinstance(x, str) else "unknown")

test["first"] = test["Name"].apply(lambda x :x.split(" ")[0] if isinstance(x, str) else "unknown")
test["last"] = test["Name"].apply(lambda x :x.split(" ")[1] if isinstance(x, str) else "unknown")

In [11]:
#欠損値補完
train["Destination"].fillna("missingPL", inplace=True)
test["Destination"].fillna("missingPL", inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2812334904.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Destination"].fillna("missingPL", inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2812334904.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [13]:
trainX = train.drop(["Cabin","Name"],axis=1)
testX = test.drop(["Cabin","Name"],axis=1)

In [15]:
trainX["Age"].fillna(trainX["Age"].mean(),inplace=True)
testX["Age"].fillna(testX["Age"].mean(),inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2132101062.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainX["Age"].fillna(trainX["Age"].mean(),inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2132101062.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [17]:
trainX["HomePlanet"].fillna("missing",inplace=True)
testX["HomePlanet"].fillna("missing",inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2492911992.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainX["HomePlanet"].fillna("missing",inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2492911992.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [19]:
trainX["CryoSleep"].fillna("None",inplace=True)
testX["CryoSleep"].fillna("None",inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\386060082.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainX["CryoSleep"].fillna("None",inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\386060082.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [21]:
trainX["VIP"].fillna("None",inplace=True)
testX["VIP"].fillna("None",inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2983853106.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainX["VIP"].fillna("None",inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\2983853106.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [23]:
trainX["RoomService"].fillna(0,inplace=True)
trainX["FoodCourt"].fillna(0,inplace=True)
trainX["ShoppingMall"].fillna(0,inplace=True)
trainX["Spa"].fillna(0,inplace=True)
trainX["VRDeck"].fillna(0,inplace=True)

testX["RoomService"].fillna(0,inplace=True)
testX["FoodCourt"].fillna(0,inplace=True)
testX["ShoppingMall"].fillna(0,inplace=True)
testX["Spa"].fillna(0,inplace=True)
testX["VRDeck"].fillna(0,inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\3053640728.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainX["RoomService"].fillna(0,inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_15900\3053640728.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [25]:
trainX["PassengerId_gg"] = trainX["PassengerId"].apply(lambda x :x.split("_")[0] if isinstance(x, str) else "unknown")

# AとBの組み合わせで同じ人が何人いるか数える
trainX["Family_num"] = trainX.groupby(["PassengerId_gg", "last"])["PassengerId"].transform("count")

# 家族：2人以上なら1、1人だけなら0
trainX["Family"] = trainX["Family_num"].apply(lambda x: 1 if x > 1 else 0)

# カウント列は不要なら削除
trainX.drop(columns=["Family_num"], inplace=True)

testX["PassengerId_gg"] = testX["PassengerId"].apply(lambda x :x.split("_")[0] if isinstance(x, str) else "unknown")
testX["Family_num"] = testX.groupby(["PassengerId_gg", "last"])["PassengerId"].transform("count")
testX["Family"] = testX["Family_num"].apply(lambda x: 1 if x > 1 else 0)
testX.drop(columns=["Family_num"], inplace=True)

In [27]:
trainX_base = trainX.drop(["PassengerId","Transported","PassengerId_gg","first","last",],axis=1)
y = trainX["Transported"]
testX_base = testX.drop(["PassengerId","PassengerId_gg","first","last",],axis=1)

In [29]:
trainX_base["num"] = trainX_base["num"].replace("unknown", 0).astype(int)
testX_base["num"] = testX_base["num"].replace("unknown", 0).astype(int)

In [45]:
from sklearn.preprocessing import LabelEncoder
# 対象のカラム一覧
label_cols = ["HomePlanet","Destination","deck","side"]

# 各カラムに LabelEncoder を適用
le_dict = {}
for col in label_cols:
    le = LabelEncoder()
    trainX_base[col] = le.fit_transform(trainX_base[col])
    testX_base[col] = le.transform(testX_base[col])


In [47]:
trainX_base = pd.get_dummies(trainX_base,dtype=int)
testX_base = pd.get_dummies(testX_base,dtype=int)

In [51]:
trainX_base.drop(["VIP_None","CryoSleep_None"],axis=1,inplace=True)
testX_base.drop(["VIP_None","CryoSleep_None"],axis=1,inplace=True)

In [53]:
#XGBモデル
xgb = XGB(use_label_encoder=False, eval_metric="auc", random_state=42)

param_grid = {'max_depth': [3, 5, 7],'learning_rate': [0.1, 0.3,0.5],'n_estimators': [50, 100,150]}

gcv = GridSearchCV(xgb, param_grid,scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [55]:
gcv.fit(trainX_base,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:03:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='auc', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=No...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [57]:
gcv_best = gcv.best_estimator_
pred = gcv_best.predict(testX_base).astype(bool)
sample["Transported"] = pred
sample.to_csv("submission_xgb_enc.csv",index=None)

In [59]:
from lightgbm import LGBMClassifier
model_LG = LGBMClassifier(use_label_encoder=False, eval_metric="auc", random_state=42)
gcv_LG = GridSearchCV(model_LG, param_grid,scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [61]:
gcv_LG.fit(trainX_base,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1640
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

GridSearchCV(cv=5,
             estimator=LGBMClassifier(eval_metric='auc', random_state=42,
                                      use_label_encoder=False),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [63]:
best_LG = gcv_LG.best_estimator_
pred_LG = best_LG.predict(testX_base).astype(bool)
sample["Transported"] = pred_LG
sample.to_csv("submission_LGBM_enc.csv",index=None)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric


In [65]:
#CatBoostモデル
from catboost import CatBoostClassifier

model_cat = CatBoostClassifier(verbose=0)
gcv_cat = GridSearchCV(model_cat, param_grid,scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [67]:
gcv_cat.fit(trainX_base,y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x000002374EC62630>,
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [69]:
best_cat = gcv_cat.best_estimator_
pred_cat = gcv_cat.predict(testX_base).astype(bool)
sample["Transported"] = pred_cat
sample.to_csv("submission_cat_enc.csv",index=None)

In [71]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
#スタッキング（XGB,LGBM,Cat）
estimators = [
    ('xgb', gcv_best),
    ('lgb', best_LG),
    ('cat', best_cat)
]

stacked_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)

In [73]:
stacked_model.fit(trainX_base,y)

C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000506 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1640
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not use

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000382 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1639
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='auc',
                                              feature_types=None,
                                              feature_weights=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_cons...
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None, ...)),
                               ('lgb',
                                LGBMClassifier(eval_metric='auc', max_depth=3,
                                               random_state=42,
                                               use_label_encoder=False)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x0000023746973CB0>)],
                   final_estimator=LogisticRegression())

In [75]:
pred_stacked = stacked_model.predict(testX_base).astype(bool)
sample["Transported"] = pred_stacked
sample.to_csv("submission_stacked_enc.csv",index=None)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
